## Install Required Libraries

In [ ]:
import sys, traceback
!{sys.executable} -m pip install gremlinpython==3.4.10
!{sys.executable} -m pip install futures
!{sys.executable} -m pip install networkx

import nest_asyncio
nest_asyncio.apply()

print(sys.version)

## Client Connection - Edit to point to correct account / DB

In [ ]:
from gremlin_python.driver import client, serializer

client = client.Client(
        '<GREMLIN ENDPOINT URL>','g', 
        username="/dbs/<DATABASE NAME>/colls/<GRAPH NAME>",
    password="<COSMOS ACCESS KEY>",
        message_serializer=serializer.GraphSONSerializersV2d0()
)


## Client Code to Execute Queries

In [ ]:
from gremlin_python.driver.protocol import GremlinServerError

cosmosdb_messages = {
    409: 'Conflict exception. You\'re probably inserting the same ID again.',
    429: 'Not enough RUs for this query. Try again.'
}

def executeGremlinQuery(gremlinQuery, message=None, params=None):
    try: 
        callback = client.submitAsync(gremlinQuery)
        if callback.result() is not None:
            return callback.result().one()
    except GremlinServerError as ex:
        status=ex.status_attributes['x-ms-status-code']
        print('There was an exception: {0}'.format(status))
        print(cosmosdb_messages[status])
        

## Create "Modern Graph" 

In [ ]:
executeGremlinQuery("g.addV('person').property(id,'v1').property('name','marko').property('age','29').property('dept','accounts').addV('person').property(id,'v2').property('name','vadas').property('age','27').property('dept','accounts').addV('software').property(id,'v3').property('name','lop').property('lang','java').property('dept','accounts').addV('person').property(id,'v4').property('name','josh').property('age','32').property('dept','accounts').addV('software').property(id,'v5').property('name','ripple').property('lang','java').property('dept','sales').addV('person').property(id,'v6').property('name','peter').property('age','35').property('dept','sales')")

executeGremlinQuery("g.V('v1').addE('created').to(g.V('v3')).property(id,'e9').property('weight','0.4')")
executeGremlinQuery("g.V('v4').addE('created').to(g.V('v5')).property(id,'e10').property('weight','1.0')")
executeGremlinQuery("g.V('v4').addE('created').to(g.V('v3')).property(id,'e11').property('weight','0.4')")
executeGremlinQuery("g.V('v6').addE('created').to(g.V('v3')).property(id,'e12').property('weight','0.2')")

executeGremlinQuery("g.V('v1').addE('knows').to(g.V('v2')).property(id,'e7').property('weight','0.4')")
executeGremlinQuery("g.V('v1').addE('knows').to(g.V('v4')).property(id,'e8').property('weight','1.0')")


result = executeGremlinQuery("g.V().count()")
print("Count of vertices: {0}".format(result))

result = executeGremlinQuery("g.E().count()")
print("Count of edges: {0}".format(result))